In [6]:
import plotly.express as px
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

In [3]:
from pyspark.sql import SparkSession
from pysparkling import H2OContext

spark = SparkSession.builder.appName("h2o_auto_ml").getOrCreate()
hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://192.168.1.139:54321 ... successful.


H2O cluster uptime:,33 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,4 days
H2O cluster name:,sparkling-water-Villoro_local-1576919476961
H2O cluster total nodes:,1
H2O cluster free memory:,905 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.28.0.1-1-2.4
 * H2O name: sparkling-water-Villoro_local-1576919476961
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,192.168.1.139,54321)
  ------------------------

  Open H2O Flow in browser: http://192.168.1.139:54321 (CMD + click in Mac OSX)

    


In [4]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=2*60)
aml.train(x=x, y=y, training_frame=train, validation_frame=test)

AutoML progress: |
10:13:16.533: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [8]:
# Optionally add extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20191221_101316,0.786824,0.555233,0.80447,0.327633,0.433842,0.188219,4184,0.186112
StackedEnsemble_BestOfFamily_AutoML_20191221_101316,0.783778,0.557955,0.800885,0.330846,0.435154,0.189359,2937,0.083792
GBM_5_AutoML_20191221_101316,0.780876,0.559708,0.797893,0.325399,0.436083,0.190168,1126,0.020318
GBM_1_AutoML_20191221_101316,0.779014,0.56159,0.79655,0.326532,0.437003,0.190971,924,0.017218
GBM_2_AutoML_20191221_101316,0.778302,0.561527,0.796277,0.328198,0.437199,0.191143,864,0.017638
GBM_3_AutoML_20191221_101316,0.776381,0.563906,0.793262,0.327534,0.438274,0.192084,968,0.019871
GBM_4_AutoML_20191221_101316,0.770768,0.570912,0.790312,0.3517,0.441681,0.195082,1154,0.020297
DRF_1_AutoML_20191221_101316,0.765142,0.580246,0.78327,0.339003,0.445292,0.198285,3154,0.028869
XRT_1_AutoML_20191221_101316,0.765105,0.582172,0.783015,0.350222,0.446013,0.198928,3757,0.030579
GBM_grid__1_AutoML_20191221_101316_model_1,0.761366,0.639299,0.778889,0.335055,0.472759,0.223501,4013,0.027509
